In [1]:
# jupyter中使用hyperparams需要添加
import sys
sys.argv = ['run.py']

根据经纬度映射出区域号.
- xlim:经度范围
- ylim:纬度范围

北京：
- lng_ld = 116.1
- lat_ld = 39.7
- lng_max = 116.7
- lat_max = 40.2

In [2]:
def get_gridID(longitude, latitude):
    # xlim = [116.1, 116.7]
    # ylim = [39.7, 40.2]
    lng_ld = 116.1
    lat_ld = 39.7
    lng_max = 116.7
    lat_max = 40.2
    grid_length = 0.0002
    grid_width = 0.0002
    # regionID = (xlim[1] - xlim[0]) // grid_length * (latitude - ylim[0]) // grid_width + (
    #         longitude - xlim[0]) // grid_length + 1
    gridID = str(int((longitude-lng_ld)/0.0002)) + "-" + str(int((latitude-lat_ld)/0.0002))
    return gridID

print(get_gridID(116.2345,39.8744))


672-871


轨迹过滤：过滤掉timeslots小于m的轨迹，过滤掉总条数少于5条的轨迹。

In [3]:
import os
file_list = sorted(os.listdir('./data/Geolife/Data'))
file_num = len(file_list)
file_list = file_list[1:]


for user_id in file_list:
    u_path = os.getcwd() + "/data" + "/Geolife" + "/Data/" + user_id + "/Trajectory"



# path = os.getcwd() + "/data" + "/Geolife" + "/Data" + "/001" + "/Trajectory"
# print(path)
# user_path = os.scandir(path)
# print(user_path)

In [4]:
output_path = './Results/size%d' % (1)
if not os.path.exists(output_path):
    # 目录不存在，进行创建操作
    os.makedirs(output_path)

  

In [5]:
import pandas as pd
import datetime
time0 = datetime.datetime.now()
start_time = datetime.datetime.now()
result_tmp = []
data_tmp = pd.DataFrame()
data_all = pd.DataFrame()  
# data_all.columns=['UID','timestamp','longitude','latitude']
for filename in file_list:
    path = "./data" + "/Geolife" + "/Data/" + filename + "/Trajectory"
    # user_path: one user's all trajectories
    user_path = os.scandir(path)
    for item in user_path:
        # path_item: one of the trajactories
        path_item = path + "/" + item.name
        with open(path_item, 'r+') as fp:
            # every item is a point
            # lat, lnt, 0, alt, date, date(日期), time（时间）
            # 39.999383,116.326916,0,134,39915.3152083333,2009-04-12,07:33:54
            for item in fp.readlines()[6::10]:
                item_list = item.strip().split(',')

                uid = filename
                lat = item_list[0]
                lnt = item_list[1]
                date = item_list[5]
                timestamp = item_list[6]

                temp = []
                temp.append(uid)
                temp.append(lat)
                temp.append(lnt)
                temp.append(date)
                temp.append(timestamp)
                result_tmp.append(temp) 
end_time = datetime.datetime.now()
print(len(result_tmp))
print("Loading Time:", str((end_time - start_time))[0:7])

2496075
Loading Time: 0:00:14


In [6]:
result_tmp = result_tmp[0:2000]
data_all = pd.DataFrame(result_tmp, columns=['uid', 'latitude', 'longitude', 'date', 'time']).dropna(axis=0)

tmp = data_all.copy(deep=True)
# 经纬度
# UID
tmp['uid'] = tmp['uid'].apply(lambda x: int(x))
# 获取日期
# 获取具体时间（24小时制）
# 获取时间编号（48个半小时中的第几个）
tmp['timeNo'] = tmp['time'].apply(lambda x: int(x[0:2]) * 2 + int(x[3:5]) // 30)
# 获得区域编号
tmp['gridID'] = tmp.apply(lambda x: get_gridID(float(x['longitude']), float(x['latitude'])), axis=1)
# tmp

```
['000', '40.000017', '116.327479', '2009-04-12', '07:33:03']
['000', '39.999383', '116.326916', '2009-04-12', '07:33:54']
['000', '39.998139', '116.327237', '2009-04-12', '07:34:44']
['000', '39.996225', '116.32658', '2009-04-12', '07:35:39']
['000', '39.995637', '116.326712', '2009-04-12', '07:36:24']
```

In [7]:
# 给区域重新编号
print("reindex regionID...")
keys = []
keys += set(tmp['gridID'])
keys = [i for n, i in enumerate(keys) if i not in keys[:n]]
print("After loading the data files, region numbers: ", len(keys))
values = [i for i in range(1, len(keys) + 1)]
tmp_dict = dict(zip(keys, values))
tmp['regionID'] = tmp.apply(lambda x: tmp_dict[x['gridID']], axis=1)
print(tmp.shape)
tmp.head(5)

reindex regionID...
After loading the data files, region numbers:  1243
(2000, 8)


,uid,latitude,longitude,date,time,timeNo,gridID,regionID
0,0,40.000017,116.327479,2009-04-12,07:33:03,15,1137-1500,1120
1,0,39.999383,116.326916,2009-04-12,07:33:54,15,1134-1496,401
2,0,39.998139,116.327237,2009-04-12,07:34:44,15,1136-1490,2
3,0,39.996225,116.32658,2009-04-12,07:35:39,15,1132-1481,227
4,0,39.995637,116.326712,2009-04-12,07:36:24,15,1133-1478,100


In [8]:
# 转换成AttnMove的数据格式：{uid, date, trajectory}
print("generate raw_pos.csv...")
tmp3 = tmp[['uid', 'date', 'regionID', 'timeNo']].copy(deep=True)
tmp3['regionID'] = tmp3['regionID'].apply(lambda x: str(x))

# aggregate the regionID for every time intervel(30 mins)
# tmp3 = tmp3.groupby(['uid', 'date', 'timeNo']).apply(lambda x: x[:])#.agg(lambda x: x.value_counts().index[0]).reset_index()
tmp3 = tmp3.groupby(['uid', 'date', 'timeNo']).agg(lambda x: x.value_counts().index[0]).reset_index()
tmp3.to_csv('./Results/tmp3.txt', sep='\t', index=False)
tmp3

generate raw_pos.csv...


,uid,date,timeNo,regionID
0,0,2008-11-13,7,150
1,0,2008-11-13,13,407
2,0,2008-11-13,15,590
3,0,2008-11-13,22,325
4,0,2008-11-13,27,386
...,...,...,...,...
130,0,2009-07-05,11,748
131,0,2009-07-05,12,235
132,0,2009-07-05,13,1167
133,0,2009-07-05,14,775


In [9]:
# 获得user_list和date_list
tmp3 = pd.read_csv('Results/tmp3.txt', sep="\t").groupby(['uid', 'date']).apply(lambda x: x[:])
user_list = tmp3['uid'].drop_duplicates().values
date_list = tmp3['date'].drop_duplicates().values
print(user_list)
print(date_list)

[0]
['2008-11-13' '2008-11-21' '2008-12-01' '2008-12-11' '2009-03-31'
 '2009-04-09' '2009-04-12' '2009-04-15' '2009-04-23' '2009-04-24'
 '2009-04-26' '2009-05-09' '2009-05-10' '2009-05-19' '2009-05-20'
 '2009-06-02' '2009-06-07' '2009-06-08' '2009-06-11' '2009-06-19'
 '2009-06-21' '2009-06-26' '2009-06-29' '2009-07-03' '2009-07-05']


In [10]:
# result_final是符合AttnMove数据格式的dataframe
result_final = pd.DataFrame()
for uid in user_list:
    for date in date_list:
        try:
            tmp4 = pd.DataFrame(tmp3.loc[(uid, date), ['timeNo', 'regionID']].values.T)
            tmp4 = tmp4.set_axis(tmp4.iloc[0], axis=1, inplace=False).drop(index=0)
            tmp4.index = pd.MultiIndex.from_product([[uid], [date]])
            result_final = result_final.append(tmp4)
        except:
            print("No record for UID=%d, date=%d" % (uid, date))
            continue

result_final        
# result_final = result_final.fillna('*')
# end_time = datetime.now()
# print("Time for converting format:", str((end_time - start_time))[0:7])
# time1 = datetime.now()
# print("Total time:", str(time1 - time0)[0:7])

0                0       1      2       3       4       5      6       7   \
0 2008-11-13    NaN     NaN    NaN     NaN     NaN     NaN    NaN   150.0   
  2008-11-21    NaN     NaN    NaN     NaN     NaN     NaN    NaN     NaN   
  2008-12-01    NaN     NaN    NaN     NaN     NaN     NaN    NaN     NaN   
  2008-12-11    NaN     NaN    NaN     NaN     NaN     NaN    NaN     NaN   
  2009-03-31    NaN  1056.0  555.0   531.0     NaN     NaN    NaN   246.0   
  2009-04-09    NaN     NaN    NaN     NaN     NaN     NaN    NaN     NaN   
  2009-04-12    NaN     NaN    NaN     NaN     NaN     NaN    NaN     NaN   
  2009-04-15  901.0     NaN    NaN     NaN     NaN     NaN    NaN     NaN   
  2009-04-23    NaN     NaN    NaN     NaN     NaN     NaN    NaN     NaN   
  2009-04-24    NaN     NaN    NaN     NaN     NaN     NaN    NaN     NaN   
  2009-04-26    NaN     NaN    NaN     NaN     NaN     NaN    NaN     NaN   
  2009-05-09    NaN     NaN    NaN     NaN     NaN     NaN    NaN     NaN   
  2009-05-10    NaN     NaN    NaN     NaN   391.0     NaN    NaN     NaN   
  2009-05-19    NaN     NaN    NaN     NaN     NaN     NaN    NaN     NaN   
  2009-05-20    NaN     NaN  814.0     NaN  1061.0   355.0    NaN     NaN   
  2009-06-02    NaN     NaN    NaN     NaN     NaN     NaN    NaN  1004.0   
  2009-06-07    NaN     NaN    NaN     NaN     NaN     NaN    NaN     NaN   
  2009-06-08    NaN     NaN    NaN     NaN     NaN     NaN    NaN     NaN   
  2009-06-11    NaN     NaN    NaN     NaN     NaN     NaN    NaN     NaN   
  2009-06-19    NaN   358.0    NaN  1075.0     NaN     NaN    NaN   512.0   
  2009-06-21    NaN     NaN    NaN     NaN     NaN     NaN    NaN     NaN   
  2009-06-26    NaN     NaN    NaN     NaN     NaN     NaN    NaN     NaN   
  2009-06-29    NaN     NaN  247.0    66.0  1190.0   681.0    NaN     NaN   
  2009-07-03  441.0  1215.0  101.0     NaN   414.0  1010.0    NaN   450.0   
  2009-07-05    NaN     NaN    NaN     NaN     NaN  1129.0  359.0  1019.0   

0                8       9   ...      26      27      28     29     34     35  \
0 2008-11-13    NaN     NaN  ...     NaN   386.0   206.0    NaN    NaN    NaN   
  2008-11-21    NaN     NaN  ...     NaN     NaN     NaN    NaN    NaN    NaN   
  2008-12-01    NaN     NaN  ...     NaN     NaN     NaN    NaN    NaN    NaN   
  2008-12-11    NaN     NaN  ...     NaN     NaN     NaN    NaN    NaN    NaN   
  2009-03-31  211.0  1120.0  ...     NaN     NaN     NaN    NaN    NaN    NaN   
  2009-04-09    NaN     NaN  ...     NaN     NaN     NaN    NaN    NaN    NaN   
  2009-04-12    NaN     NaN  ...  1163.0   314.0   495.0    NaN  604.0    NaN   
  2009-04-15    NaN     NaN  ...     NaN     NaN     NaN    NaN    NaN    NaN   
  2009-04-23    NaN     NaN  ...     NaN     NaN     NaN    NaN    NaN    NaN   
  2009-04-24    NaN     NaN  ...     NaN     NaN     NaN    NaN  833.0    NaN   
  2009-04-26    NaN     NaN  ...     NaN     NaN     NaN    NaN    NaN   16.0   
  2009-05-09    NaN     NaN  ...     NaN     NaN     NaN    NaN    NaN    NaN   
  2009-05-10    NaN     NaN  ...     NaN     NaN     NaN    NaN    NaN    NaN   
  2009-05-19    NaN     NaN  ...     NaN     NaN     NaN    NaN    NaN  732.0   
  2009-05-20    NaN     NaN  ...     NaN     NaN     NaN    NaN    NaN    NaN   
  2009-06-02  854.0   240.0  ...     NaN     NaN     NaN    NaN    NaN    NaN   
  2009-06-07    NaN     NaN  ...     NaN     NaN     NaN    NaN    NaN    NaN   
  2009-06-08    NaN     NaN  ...     NaN  1183.0  1176.0    NaN    NaN    NaN   
  2009-06-11    NaN     NaN  ...     NaN     NaN     NaN    NaN    NaN    NaN   
  2009-06-19  246.0     NaN  ...     NaN     NaN   408.0  408.0    NaN    NaN   
  2009-06-21    NaN     NaN  ...     4.0   352.0  1151.0  750.0    NaN    NaN   
  2009-06-26    NaN     NaN  ...  1199.0     NaN     NaN    NaN    NaN    NaN   
  2009-06-29    NaN     NaN  ...     NaN     NaN     NaN    NaN    NaN    NaN   
  2009-07-03  476.0    29.0  ...     NaN     NaN     